In [2]:
import time
load("kraus_tables.sage")

In [3]:
DB = CremonaDatabase(); DB
p = 2
kraus = kraus_table_Q2

In [ ]:
# need to impose bad additive reduction and potentially good reduction (j valuation >= 0)
# then compute monodromy type and record Kodaira type

def mtype_ktype(conductor_range):
    t0 = time.time()    
    label_type_tuples = []
    no_curves = 0
    
    for E in DB.iter(conductor_range):
        no_curves += 1
        if valuation(E.j_invariant(), p) >= 0:
            # run Tate's algorithm to get reduction type
            LD = E.local_data(p)
            if LD.has_additive_reduction():
                # run Kraus's algorithm to determine monodromy type (returns group order)
                m_type = kraus(E)
                label_type_tuples.append((E.cremona_label(), 
                                          LD.kodaira_symbol(), 
                                          m_type))
    t1 = time.time()
    
    print('{} curves in conductor range.'.format(no_curves))
    print('{} curves with bad potentially good reduction.'.format(len(label_type_tuples)))
    print("Time: {}".format(t1-t0))
    return label_type_tuples

In [4]:
label_type_tuples = []

t0 = time.time()
for i in range(5): # adjust this as needed; set to 40 for full table, at present
    (N0, N1) = 10000*i, 10000*(i+1)
    conductor_range = iter(range(N0, N1, 2))
    print('Conductor range: {}'.format((N0, N1, p)))
    label_type_tuples += mtype_ktype(conductor_range)
    print
t1 = time.time()

print('TOTAL RUNTIME: {}'.format(t1-t0))  # TOTAL RUNTIME: 2427.10351396 on my Dell Latitude E5270

Conductor range: (0, 10000, 2)
50462 curves in conductor range.
20979 curves with bad potentially good reduction.
Time: 34.561537981

Conductor range: (10000, 20000, 2)
53983 curves in conductor range.
22692 curves with bad potentially good reduction.
Time: 37.1490769386

Conductor range: (20000, 30000, 2)
53554 curves in conductor range.
22560 curves with bad potentially good reduction.
Time: 35.5450279713

Conductor range: (30000, 40000, 2)
53121 curves in conductor range.
22231 curves with bad potentially good reduction.
Time: 37.4674530029

Conductor range: (40000, 50000, 2)
53480 curves in conductor range.
22809 curves with bad potentially good reduction.
Time: 37.2663981915

TOTAL RUNTIME: 181.996042013


In [9]:
# You don't want to run this unless you want to recompute over the entire Cremona database
# objfile = os.path.join('./', '2-ktype_mtype_data.sobj')
# save(label_type_tuples, objfile)

In [3]:
ktype_given_mtype = dict.fromkeys([2, 3, 4, 6, 8, 24], [])
for (_, ktype, mtype) in label_type_tuples:
    if ktype not in ktype_given_mtype[mtype]:
        ktype_given_mtype[mtype] = ktype_given_mtype[mtype] + [ktype]
ktype_given_mtype

{2: [II*, I4*, II, I8*],
 3: [IV*, IV],
 4: [III, III*],
 6: [I0*, II, I4*, II*],
 8: [I3*, III, I0*, I2*, I5*, II, III*],
 24: [I1*, III*, III, II*, I0*, I2*, II, I3*, I4*, I6*, I7*]}

In [26]:
first_examples = []
for (label, ktype, mtype) in label_type_tuples:
    if (ktype, mtype) not in [e[1:] for e in first_examples]:
        first_examples += [(label, ktype, mtype)]

objfile = os.path.join('./', '2-ktype_mtype_first_examples.sobj')
save(first_examples, objfile)
first_examples = load(objfile)

In [8]:
def k_type_to_cpt_group(symbol):
    if symbol in [KodairaSymbol('II'), KodairaSymbol('II*')]:
        return "0"
    elif symbol in [KodairaSymbol('III'), KodairaSymbol('III*')]:
        return "Z/2"
    elif symbol in [KodairaSymbol('IV'), KodairaSymbol('IV*')]:
        return "Z/3"
    elif str(symbol)[-1] == '*':
        if int(str(symbol)[1:-1]) % 2 == 0:
            return "Z/2 x Z/2"
        else:
            return "Z/4"
    else:
        print "Invalid reduction type"

In [9]:
for m_type in ktype_given_mtype:
    cpt_gps = map(lambda t: k_type_to_cpt_group(t), ktype_given_mtype[m_type])
    cpt_gps = list(set(cpt_gps))
    print(m_type, cpt_gps)

(2, ['0', 'Z/2 x Z/2'])
(3, ['Z/3'])
(4, ['Z/2'])
(6, ['Z/2 x Z/2', '0'])
(8, ['Z/2 x Z/2', 'Z/2', 'Z/4', '0'])
(24, ['0', 'Z/2', 'Z/4', 'Z/2 x Z/2'])


In [10]:
# remarks:
# McCallum proves exponent of inertial monodromy kills component group
# exponents: 2, 3, 4, 6, 8, 24 |-> 2, 3, 4, 6, 4, 12
#
# for C_2, the possible component group are: 0, C_2, C_2 x C_2,
#        but we don't see C_2
# note: in this case, monodromy group smaller than component group
# note: I think Dokchitser paper shows this is the only situation where potentially ordinary
#       reduction is possible.
#
# for C_3, possibilities are 0 and C_3, 
#        but we don't see 0
#
# for C_4, possibilities are 0, C_2, C_4, C_2 x C_2,
#        but we ONLY see one of these, C_2
#
# for C_6, possibilities are 0, C_2, C_3, C_2 x C_2,
#        but we don't see C_2 or C_3
# note: C_2 x C_2 is not a subquotient of C_6
# note: Dokchitsers show this is potentially supersingular
#
# for Q8, the possiblities are 0, C_2, C_4, C_2 x C_2, and we see all of these
# 
# for SL_2(F_3), the possiblities are 0, C_2, C_4, C_2 x C_2, and wee see all of these